In [3]:
import pandas as pd
import json

def ingest_df(file_path, file_type='csv', delimiter=','):
    """
    Ingests df from CSV or JSON based on file type.
    
    :param file_path: str, path to the file
    :param file_type: str, type of file ('csv' or 'json')
    :param delimiter: str, delimiter for CSV files (default is comma)
    :return: dfFrame with ingested df
    """
    
    if file_type == 'csv':
        # Ingest CSV file with configurable delimiter
        df = pd.read_csv(file_path, delimiter=delimiter)
        
    elif file_type == 'json':
        # Ingest JSON file
        with open(file_path, 'r') as file:
            df = json.load(file)
        df = pd.dfFrame(df)
    
    else:
        raise ValueError(f"Unsupported file type: {file_type}")
    
    return df

# Example usage
df = ingest_df('sales_performance_data.csv', file_type='csv')
# df = ingest_df('/path/to/file.json', file_type='json')


In [4]:
df.head()

,employee_id,employee_name,created,dated,lead_taken,tours_booked,applications,tours_per_lead,apps_per_tour,apps_per_lead,...,fri_text,sat_text,sun_text,mon_call,tue_call,wed_call,thur_call,fri_call,sat_call,sun_call
0,183,Camilla Ali,2022-07-26 19:00:12.644,2022-07-26,44,2,2,4.5,100.0,4.5,...,11,5,86,0,3,2,6,0,0,13
1,185,Maaz Brown,2022-07-26 19:00:12.644,2022-07-26,27,1,0,3.7,0.0,0.0,...,27,28,21,2,1,0,2,3,1,3
2,186,Jim Lee,2022-07-26 19:00:12.644,2022-07-26,86,5,1,5.8,20.0,1.2,...,10,14,0,0,2,0,0,0,0,0
3,173,Alina Victor,2022-07-26 19:00:12.644,2022-07-26,47,8,4,17.0,50.0,8.5,...,10,20,6,4,5,13,1,6,10,1
4,158,Mary Ogbewele,2022-07-26 19:00:12.644,2022-07-26,56,6,3,10.7,50.0,5.4,...,56,52,13,5,12,0,3,11,0,2


In [5]:
df.isnull().sum()

employee_id               0
employee_name             9
created                   0
dated                     0
lead_taken                0
tours_booked              0
applications              0
tours_per_lead            0
apps_per_tour             0
apps_per_lead             0
revenue_confirmed         0
revenue_pending           0
revenue_runrate           0
tours_in_pipeline         0
avg_deal_value_30_days    0
avg_close_rate_30_days    0
estimated_revenue         0
tours                     0
tours_runrate             0
tours_scheduled           0
tours_pending             0
tours_cancelled           0
mon_text                  0
tue_text                  0
wed_text                  0
thur_text                 0
fri_text                  0
sat_text                  0
sun_text                  0
mon_call                  0
tue_call                  0
wed_call                  0
thur_call                 0
fri_call                  0
sat_call                  0
sun_call            

In [6]:
df['dated'] = pd.to_datetime(df['dated'], errors='coerce')

In [7]:
df['created'] = pd.to_timedelta(df['created'], errors='coerce')

In [8]:
numeric_columns = ['lead_taken', 'tours_booked', 'applications', 'revenue_confirmed', 'revenue_pending', 'revenue_runrate']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [9]:
df['tour_to_lead_conversion'] = df['tours_booked'] / df['lead_taken']
df['application_to_tour_conversion'] = df['applications'] / df['tours_booked']


In [10]:
df['revenue_per_tour'] = df['revenue_confirmed'] / df['tours_booked']


In [11]:
# Summarize data for the entire team
team_summary = df.agg({
    'lead_taken': 'sum',
    'tours_booked': 'sum',
    'applications': 'sum',
    'revenue_confirmed': 'sum',
    'revenue_pending': 'sum',
    'revenue_runrate': 'mean',  
})

print(team_summary)


lead_taken           1.434000e+05
tours_booked         8.061000e+03
applications         3.908000e+03
revenue_confirmed    2.131691e+06
revenue_pending      1.731541e+06
revenue_runrate      2.694492e+03
dtype: float64


In [12]:
# Generate a text summary for each representative
def generate_rep_summary(row):
    return f"""Employee: {row['employee_name']}
    Leads Taken: {row['lead_taken']}
    Tours Booked: {row['tours_booked']}
    Applications: {row['applications']}
    Revenue Confirmed: {row['revenue_confirmed']}
    Revenue Pending: {row['revenue_pending']}
    Conversion Rate (Tours to Leads): {row['tour_to_lead_conversion']:.2f}
    """

# Apply to each row
df['summary'] = df.apply(generate_rep_summary, axis=1)
print(df['summary'])


0       Employee: Camilla Ali\n    Leads Taken: 44\n  ...
1       Employee: Maaz Brown\n    Leads Taken: 27\n   ...
2       Employee: Jim Lee\n    Leads Taken: 86\n    To...
3       Employee: Alina Victor\n    Leads Taken: 47\n ...
4       Employee: Mary Ogbewele\n    Leads Taken: 56\n...
                              ...                        
2531    Employee: Travis Reed\n    Leads Taken: 3\n   ...
2532    Employee: nan\n    Leads Taken: 0\n    Tours B...
2533    Employee: Richard Crown\n    Leads Taken: 25\n...
2534    Employee: Rebecca Zara\n    Leads Taken: 5\n  ...
2535    Employee: Mary Farley\n    Leads Taken: 0\n   ...
Name: summary, Length: 2536, dtype: object


In [13]:
team_summary_text = f"""
The team took a total of {team_summary['lead_taken']} leads, booked {team_summary['tours_booked']} tours, and confirmed ${team_summary['revenue_confirmed']} in revenue. The average revenue run rate is ${team_summary['revenue_runrate']}.
"""
print(team_summary_text)



The team took a total of 143400.0 leads, booked 8061.0 tours, and confirmed $2131691.0 in revenue. The average revenue run rate is $2694.4921135646687.



In [14]:
from langchain_huggingface import HuggingFacePipeline
# Load the model using HuggingFacePipeline
llm = HuggingFacePipeline.from_model_id(
    model_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 100,
        "top_k": 50,
        "temperature": 0.1,
    },
)

# Function to generate insights from the LLM
def generate_insights(data: pd.DataFrame) -> str:
    # Convert DataFrame to a string representation for the prompt
    formatted_data = data.to_string(index=False)
    prompt = f"Analyze the following sales data and provide feedback:\n{formatted_data}\nProvide qualitative feedback and actionable insights."
    
    # Invoke the LLM with the generated prompt
    response = llm.invoke(prompt)
    return response

# Example usage
df = ingest_df('sales_performance_data.csv', file_type='csv')

# Data preprocessing
df['dated'] = pd.to_datetime(df['dated'], errors='coerce')
df['created'] = pd.to_timedelta(df['created'], errors='coerce')
numeric_columns = ['lead_taken', 'tours_booked', 'applications', 'revenue_confirmed', 'revenue_pending', 'revenue_runrate']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df['tour_to_lead_conversion'] = df['tours_booked'] / df['lead_taken']
df['application_to_tour_conversion'] = df['applications'] / df['tours_booked']
df['revenue_per_tour'] = df['revenue_confirmed'] / df['tours_booked']

# Generate insights from the LLM
insights = generate_insights(df)
print(insights)